In [1]:
import json
import time
import datetime
import random
import numpy as np

import TDN_CraneSim_PyIF

print(TDN_CraneSim_PyIF.version())

[0, 8, 1]


In [2]:

PICKED_KEYS = ["step", "turning_encoder_angle_deg", "turning_encoder_acquisition_time"]

def print_state(state_src, keys = PICKED_KEYS):
    """
    craneのstateから任意の情報を取り出して出力する、デバッグ用のメソッドです。

    Parameters
    ----------
    state_src : dict
        craneから取得したstate
    keys : list[string]
        着目するkey
    """

    state_ret = {}
    for key in keys:
        state_ret[key] = state_src[key]

    t = datetime.datetime.now()
    #print( "  [{}] state = {}".format( t.strftime('%H:%M:%S.%f')[:-3],  state_ret) )
    print( state_ret )


def execute( param_sim, param_crane ):

    ###################################################################
    # シミュレーションの開始準備
    ###################################################################

    # シミュレータを構築
    # set_param はJSON文字列での設定が必要なので注意して下さい。
    crane = TDN_CraneSim_PyIF.CraneSimulator( json.dumps(param_sim), json.dumps(param_crane) )

    # クレーンの状態を取得。
    state = crane.get_crane_state()


    ###################################################################
    # シミュレーションの実行と状態確認
    ###################################################################

    # シミュレーションの実行をスタートします。
    crane.start()

    # ここでは、旋回角度が60度になるまで旋回します。
    # なお旋回量は state['turning_encoder_angle_deg'] に格納
    while( state['turning_encoder_angle_deg'] < 60 ) :

        # ステップモードの場合、明示的にシミュレーションのステップを進める必要があります。
        # 1ステップ 0.01s なので、今回は 10*0.01s = 0.1s だけシミュレーションを進めます。
        crane.step(10)

        # 適当なレバー操作量を設定しています。
        # レバー操作量の値域は ( -1 ～ 1 ) に変更しているのでご注意下さい。
        crane.set_action_value(0.8)

        # 現時点でのクレーン状態を取得します。
        state = crane.get_crane_state()

        # デバッグ用の出力を行います。
        print_state(state)

    # ループを抜けたらレバー操作量を0にしてし10秒待ちます。
    crane.set_action_value(0)
    stop_step = state['step'] + 100 * 10 # 100Hz で 10秒
    while ( state['step'] <= stop_step ):
        crane.step(10)
        state = crane.get_crane_state()
        print_state(state)

    # シミュレーションを終了します。
    crane.stop()

In [3]:
# シミュレータの動作設定
param_sim = {
        'use_gui':True,
        'use_noise':True,
        'noise_randomize_seed':0,
        'use_tawami':True
}
# クレーンの動作設定
param_crane = {
        'init_kifuku_deg':60,
        'weight_t':1,
        'init_yure_senkai_m':0,
        'param_randomize_seed':0
} 

In [4]:
execute(param_sim, param_crane)

{'step': 11, 'turning_encoder_angle_deg': -4.605719380736758e-12, 'turning_encoder_acquisition_time': 110.0}
{'step': 21, 'turning_encoder_angle_deg': 0.022746256452402427, 'turning_encoder_acquisition_time': 210.0}
{'step': 31, 'turning_encoder_angle_deg': 0.2184597928003395, 'turning_encoder_acquisition_time': 310.0}
{'step': 41, 'turning_encoder_angle_deg': 0.697379727377284, 'turning_encoder_acquisition_time': 410.0}
{'step': 51, 'turning_encoder_angle_deg': 1.4586898908609194, 'turning_encoder_acquisition_time': 510.0}
{'step': 61, 'turning_encoder_angle_deg': 2.410110715853243, 'turning_encoder_acquisition_time': 610.0}
{'step': 71, 'turning_encoder_angle_deg': 3.451623378066936, 'turning_encoder_acquisition_time': 710.0}
{'step': 81, 'turning_encoder_angle_deg': 4.551307796753407, 'turning_encoder_acquisition_time': 810.0}
{'step': 91, 'turning_encoder_angle_deg': 5.634090104273267, 'turning_encoder_acquisition_time': 910.0}
{'step': 101, 'turning_encoder_angle_deg': 6.743028693